In [ ]:
# Project #1 Juypter Notebook
# Created by Melinda Eudy, Cameron Farquhar, Markita Francis, Luis Rojas, Kelly Rose
# Project : 2019 Crime Data Used from several resources ---UCR Crime Rates by County, US Census, and 

In [129]:
# Dependencies and Setup *******Review before final presentation and remove what is not needed*********
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import scipy.stats as st
from datetime import date
from functools import reduce
import json
import ctypes  # An included library with Python install.
import datetime

# Import API key
#from config import apikey

In [130]:
# Part 1 - open UCR Crime Rates by County.csv file
#----------open 37059-Codebook County Codes.csv file and merge into first file to obtain County Names 
UCR_Crime = "Resources/UCR_Crime_Rates_by_County.csv"

# Read file and store in Pandas data frame
UCR_Crime_df = pd.read_csv(UCR_Crime)
UCR_Crime_df.head()
#UCR_Crime_df.count() ----all columns had same count

,STUDYNO,EDITION,PART,IDNO,FIPS_ST,FIPS_CTY,SPOPARST,SPOPCRIM,CPOPCRIM,AG_ARRST,...,SVIOL,SPROPERTY,SMURDER,SRAPE,SROBBERY,SAGASSLT,SBURGLRY,SLARCENY,SMVTHEFT,SARSON
0,9999,1,8,1,1,1,4863300,4816193,54499,3,...,0,0,0,0,0,0,0,0,0,0
1,9999,1,8,2,1,3,4863300,4816193,207584,14,...,0,0,0,0,0,0,0,0,0,0
2,9999,1,8,3,1,5,4863300,4816193,25778,5,...,0,0,0,0,0,0,0,0,0,0
3,9999,1,8,4,1,7,4863300,4816193,22474,4,...,0,0,0,0,0,0,0,0,0,0
4,9999,1,8,5,1,9,4863300,4816193,57565,6,...,0,0,0,0,0,0,0,0,0,0


In [131]:
# FIPS-ST is the first number of the Code, FIPS_City is the last three numbers of the code.
#-----Make FIPS_City three digits wide (adding zeros in front) and combine the two columns
UCR_Crime_df["FIPS_ST"] = UCR_Crime_df["FIPS_ST"].apply(str)
UCR_Crime_df["FIPS_CTY"] = UCR_Crime_df["FIPS_CTY"].apply(str)
UCR_Crime_df["FIPS_CTY"] = UCR_Crime_df["FIPS_CTY"].apply(lambda x: x.zfill(3))
UCR_Crime_df["CODE"] = UCR_Crime_df[["FIPS_ST", "FIPS_CTY"]].agg("".join, axis=1)
UCR_Crime_df.head()

,STUDYNO,EDITION,PART,IDNO,FIPS_ST,FIPS_CTY,SPOPARST,SPOPCRIM,CPOPCRIM,AG_ARRST,...,SPROPERTY,SMURDER,SRAPE,SROBBERY,SAGASSLT,SBURGLRY,SLARCENY,SMVTHEFT,SARSON,CODE
0,9999,1,8,1,1,001,4863300,4816193,54499,3,...,0,0,0,0,0,0,0,0,0,1001
1,9999,1,8,2,1,003,4863300,4816193,207584,14,...,0,0,0,0,0,0,0,0,0,1003
2,9999,1,8,3,1,005,4863300,4816193,25778,5,...,0,0,0,0,0,0,0,0,0,1005
3,9999,1,8,4,1,007,4863300,4816193,22474,4,...,0,0,0,0,0,0,0,0,0,1007
4,9999,1,8,5,1,009,4863300,4816193,57565,6,...,0,0,0,0,0,0,0,0,0,1009


In [132]:
# Finalize file for merging
#-----remove columns: STUDYNO, EDITION, PART, FLIPS_ST, FLIPS_CTY
#-----move Code to the front of the file - ensure that it is a int64.
UCR_Crime_df.drop(columns = ["STUDYNO", "EDITION", "PART", "FIPS_ST", "FIPS_CTY"], inplace = True)
UCR_Crime_df["CODE"] = UCR_Crime_df["CODE"].apply("int64")
pd.set_option("precision",0)
collist = UCR_Crime_df.columns.tolist()
collist.insert(0, collist.pop(collist.index("CODE")))
UCR_Crime_df = UCR_Crime_df.reindex(columns = collist)
UCR_Crime_df.head()

,CODE,IDNO,SPOPARST,SPOPCRIM,CPOPCRIM,AG_ARRST,AG_OFF,COVIND,SVIOL,SPROPERTY,SMURDER,SRAPE,SROBBERY,SAGASSLT,SBURGLRY,SLARCENY,SMVTHEFT,SARSON
0,1001,1,4863300,4816193,54499,3,3,100,0,0,0,0,0,0,0,0,0,0
1,1003,2,4863300,4816193,207584,14,14,100,0,0,0,0,0,0,0,0,0,0
2,1005,3,4863300,4816193,25778,5,4,94,0,0,0,0,0,0,0,0,0,0
3,1007,4,4863300,4816193,22474,4,5,96,0,0,0,0,0,0,0,0,0,0
4,1009,5,4863300,4816193,57565,6,6,98,0,0,0,0,0,0,0,0,0,0


In [133]:
UCR_Codes = "Resources/Codebook_County_Codes.csv"

# Read file and store in Pandas data frame
UCR_Codes_df = pd.read_csv(UCR_Codes)
#UCR_Codes_df.count()   --- all columns had the same count
UCR_Codes_df.head()

,State,Code,County,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Alabama,1001,Autauga,NaN,NaN,NaN
1,Alabama,1003,Baldwin,NaN,NaN,NaN
2,Alabama,1005,Barbour,NaN,NaN,NaN
3,Alabama,1007,Bibb,NaN,NaN,NaN
4,Alabama,1009,Blount,NaN,NaN,NaN


In [134]:
# Before mergining data:
# ------Remove Unnamed:3 - Unnamed:5
# ------Remove decimal in Code
#-------Rename Code to CODE
UCR_Codes_df.drop(columns = ["Unnamed: 3", "Unnamed: 4", "Unnamed: 5"], inplace = True)
UCR_Codes_df.rename(columns={"Code":"CODE", "State":"STATE", "County":"COUNTY"}, inplace = True)
UCR_Codes_df["CODE"] = UCR_Codes_df["CODE"].apply("int64")
pd.set_option("precision",0)
UCR_Codes_df.head()

,STATE,CODE,COUNTY
0,Alabama,1001,Autauga
1,Alabama,1003,Baldwin
2,Alabama,1005,Barbour
3,Alabama,1007,Bibb
4,Alabama,1009,Blount


In [135]:
# Merge Codebook_County_Codes.csv into UCR_Crime_Rates_by_County.csv
merged_UCR_Crime_df = pd.merge(UCR_Codes_df, UCR_Crime_df,on="CODE", how="right")
merged_UCR_Crime_df.head(15)

,STATE,CODE,COUNTY,IDNO,SPOPARST,SPOPCRIM,CPOPCRIM,AG_ARRST,AG_OFF,COVIND,SVIOL,SPROPERTY,SMURDER,SRAPE,SROBBERY,SAGASSLT,SBURGLRY,SLARCENY,SMVTHEFT,SARSON
0,Alabama,1001,Autauga,1,4863300,4816193,54499,3,3,100,0,0,0,0,0,0,0,0,0,0
1,Alabama,1003,Baldwin,2,4863300,4816193,207584,14,14,100,0,0,0,0,0,0,0,0,0,0
2,Alabama,1005,Barbour,3,4863300,4816193,25778,5,4,94,0,0,0,0,0,0,0,0,0,0
3,Alabama,1007,Bibb,4,4863300,4816193,22474,4,5,96,0,0,0,0,0,0,0,0,0,0
4,Alabama,1009,Blount,5,4863300,4816193,57565,6,6,98,0,0,0,0,0,0,0,0,0,0
5,Alabama,1011,Bullock,6,4863300,4816193,10634,2,2,95,0,0,0,0,0,0,0,0,0,0
6,Alabama,1013,Butler,7,4863300,4816193,19952,4,4,97,0,0,0,0,0,0,0,0,0,0
7,Alabama,1015,Calhoun,8,4863300,4816193,114022,9,10,100,0,0,0,0,0,0,0,0,0,0
8,Alabama,1017,Chambers,9,4863300,4816193,34047,4,4,98,0,0,0,0,0,0,0,0,0,0
9,Alabama,1019,Cherokee,10,4863300,4816193,25774,4,5,99,0,0,0,0,0,0,0,0,0,0


In [187]:
# Bring in data from the US census per couounty
# adding median income per county
response = requests.get("https://api.census.gov/data/2019/acs/acs1?get=NAME,B06011_001E&for=county:*").json()
# locate the first row and make it the column name and then drop row 0
income_df = pd.DataFrame(response)
income_df.columns = income_df.iloc[0]
income_df = income_df.drop([1])

# delete and null info and reset the index
income_df = income_df.dropna(how='any')
income_df.reset_index()
income_df

,NAME,B06011_001E,state,county
0,NAME,B06011_001E,state,county
2,"Calhoun County, Alabama",24513,01,015
3,"Cullman County, Alabama",25523,01,043
4,"DeKalb County, Alabama",22076,01,049
5,"Elmore County, Alabama",31124,01,051
...,...,...,...,...
822,"Waukesha County, Wisconsin",42434,55,133
823,"Winnebago County, Wisconsin",32222,55,139
824,"Wood County, Wisconsin",31040,55,141
825,"Laramie County, Wyoming",36805,56,021


In [188]:
#replace the integer column names with values in row 0 and convert variable B06011_001E to Median Income
income_df.columns = income_df.iloc[0]
income_df = income_df.drop([0])
income_df.reset_index()

,index,NAME,B06011_001E,state,county
0,2,"Calhoun County, Alabama",24513,01,015
1,3,"Cullman County, Alabama",25523,01,043
2,4,"DeKalb County, Alabama",22076,01,049
3,5,"Elmore County, Alabama",31124,01,051
4,6,"Etowah County, Alabama",23297,01,055
...,...,...,...,...,...
820,822,"Waukesha County, Wisconsin",42434,55,133
821,823,"Winnebago County, Wisconsin",32222,55,139
822,824,"Wood County, Wisconsin",31040,55,141
823,825,"Laramie County, Wyoming",36805,56,021


In [189]:
# create new columns for County and State
# ----Remove Columns: NAME, state, county
#-----Change column names to match merged UCR Crime data
#-----Remove word "County" in County name
income_df['County Name'] = income_df['NAME'].str.split(',').apply(lambda x: x[0])
income_df['State Name'] = income_df['NAME'].str.split(',').apply(lambda x: x[-1])
income_df['County Name'] = income_df['NAME'].str.split(' ').apply(lambda x: x[0])
income_df.drop(columns = ["NAME", "state", "county"], inplace = True)
income_df.rename(columns={"County Name":"COUNTY", "State Name":"STATE", "B06011_001E":"MEDIAN INCOME"}, inplace = True)
income_df.head(15)

,MEDIAN INCOME,COUNTY,STATE
2,24513,Calhoun,Alabama
3,25523,Cullman,Alabama
4,22076,DeKalb,Alabama
5,31124,Elmore,Alabama
6,23297,Etowah,Alabama
7,26035,Houston,Alabama
8,30732,Jefferson,Alabama
9,25048,Lauderdale,Alabama
10,27057,Lee,Alabama
11,29369,Limestone,Alabama


In [190]:
# Merge Census Median Income into UCR_Crime_Rates_by_County.csv
# Move Median Income to front of chart
merged2_UCR_Crime_df = pd.merge(merged_UCR_Crime_df, income_df, on=["STATE", "COUNTY"])
collist = merged2_UCR_Crime_df.columns.tolist()
collist.insert(4, collist.pop(collist.index("MEDIAN INCOME")))
merged2_UCR_Crime_df = merged2_UCR_Crime_df.reindex(columns = collist)

# delete and null info and reset the index
#merged2_UCR_Crime_df = merged2_UCR_Crime_df.dropna(how='any')
#merged2_UCR_Crime_df.reset_index()
merged2_UCR_Crime_df.head(15)

,STATE,CODE,COUNTY,IDNO,MEDIAN INCOME,SPOPARST,SPOPCRIM,CPOPCRIM,AG_ARRST,AG_OFF,...,SVIOL,SPROPERTY,SMURDER,SRAPE,SROBBERY,SAGASSLT,SBURGLRY,SLARCENY,SMVTHEFT,SARSON


In [191]:
# Bring in data from the US census per county
#------population density per county
#------median income per county
response = requests.get(f'https://api.census.gov/data/2019/pep/population?get=DENSITY,POP,NAME&for=county:*').json()

# locate the first row and make it the column name and then drop row 0
county_dens_df = pd.DataFrame(response)
county_dens_df.columns = county_dens_df.iloc[0]
county_dens_df = county_dens.drop([1])

# delete and null info and reset the index
county_dens_df = county_dens_df.dropna(how='any')
county_dens_df.reset_index()
county_dens_df

,DENSITY,POP,NAME,state,county,County Name,State Name
2,14555.93898300000000,672391,"Hudson County, New Jersey",34,017,Hudson County,New Jersey
3,290.70129177000000,124371,"Hunterdon County, New Jersey",34,019,Hunterdon County,New Jersey
4,1067.00355890000000,491845,"Morris County, New Jersey",34,027,Morris County,New Jersey
5,13.09323114300000,71367,"McKinley County, New Mexico",35,031,McKinley County,New Mexico
6,39.55353872600000,146748,"Sandoval County, New Mexico",35,043,Sandoval County,New Mexico
...,...,...,...,...,...,...,...
3216,8.02155238020000,42243,"Okanogan County, Washington",53,047,Okanogan County,Washington
3217,20.86571669800000,47935,"Kittitas County, Washington",53,037,Kittitas County,Washington
3218,120.22256331000000,204390,"Benton County, Washington",53,005,Benton County,Washington
3219,776.85533336000000,488241,"Clark County, Washington",53,011,Clark County,Washington


In [192]:
# create new columns for County and State
# ----Remove Columns: NAME, state, county
#-----Change column names to match merged UCR Crime data
#-----Remove word "County" in County name
county_dens_df['County Name'] = county_dens_df['NAME'].str.split(',').apply(lambda x: x[0])
county_dens_df['State Name'] = county_dens_df['NAME'].str.split(',').apply(lambda x: x[-1])
county_dens_df['County Name'] = county_dens_df['NAME'].str.split(' ').apply(lambda x: x[0])
county_dens_df.drop(columns = ["NAME", "state", "county"], inplace = True)
county_dens_df.rename(columns={"County Name":"COUNTY", "State Name":"STATE"}, inplace = True)
county_dens_df.head()

,DENSITY,POP,COUNTY,STATE
2,14555.93898300000000,672391,Hudson,New Jersey
3,290.70129177000000,124371,Hunterdon,New Jersey
4,1067.00355890000000,491845,Morris,New Jersey
5,13.09323114300000,71367,McKinley,New Mexico
6,39.55353872600000,146748,Sandoval,New Mexico


In [158]:
# Merge Codebook_County_Codes.csv into UCR_Crime_Rates_by_County.csv
# Move Density and Population to front of chart
mergede_UCR_Crime_df = pd.merge(merged_UCR_Crime_df, county_dens_df, on=["STATE"], how = "left")
collist = mergede_UCR_Crime_df.columns.tolist()
collist.insert(4, collist.pop(collist.index("DENSITY")))
collist.insert(5, collist.pop(collist.index("POP")))
mergede_UCR_Crime_df = mergede_UCR_Crime_df.reindex(columns = collist)

# delete and null info and reset the index
#merged2_UCR_Crime_df = merged2_UCR_Crime_df.dropna(how='any')
#merged2_UCR_Crime_df.reset_index()
merged2_UCR_Crime_df.head(15)


,STATE,CODE,COUNTY_x,IDNO,DENSITY,POP,SPOPARST,SPOPCRIM,CPOPCRIM,AG_ARRST,...,SPROPERTY,SMURDER,SRAPE,SROBBERY,SAGASSLT,SBURGLRY,SLARCENY,SMVTHEFT,SARSON,COUNTY_y
0,Alabama,1001,Autauga,1,NaN,NaN,4863300,4816193,54499,3,...,0,0,0,0,0,0,0,0,0,NaN
1,Alabama,1003,Baldwin,2,NaN,NaN,4863300,4816193,207584,14,...,0,0,0,0,0,0,0,0,0,NaN
2,Alabama,1005,Barbour,3,NaN,NaN,4863300,4816193,25778,5,...,0,0,0,0,0,0,0,0,0,NaN
3,Alabama,1007,Bibb,4,NaN,NaN,4863300,4816193,22474,4,...,0,0,0,0,0,0,0,0,0,NaN
4,Alabama,1009,Blount,5,NaN,NaN,4863300,4816193,57565,6,...,0,0,0,0,0,0,0,0,0,NaN
5,Alabama,1011,Bullock,6,NaN,NaN,4863300,4816193,10634,2,...,0,0,0,0,0,0,0,0,0,NaN
6,Alabama,1013,Butler,7,NaN,NaN,4863300,4816193,19952,4,...,0,0,0,0,0,0,0,0,0,NaN
7,Alabama,1015,Calhoun,8,NaN,NaN,4863300,4816193,114022,9,...,0,0,0,0,0,0,0,0,0,NaN
8,Alabama,1017,Chambers,9,NaN,NaN,4863300,4816193,34047,4,...,0,0,0,0,0,0,0,0,0,NaN
9,Alabama,1019,Cherokee,10,NaN,NaN,4863300,4816193,25774,4,...,0,0,0,0,0,0,0,0,0,NaN
